In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# import the necessary packages
import numpy as np
import os
from PIL import Image, ImageOps
from sys import stdout

### Create pair of images

In [ ]:
original_basepath = '/content/drive/My Drive/DL project/data_plain/r/r'
def get_edited_folders():
  return ['bordered/#000', 'bordered/#fff', 'cropped/center', 'cropped/left_half', 'cropped/lower_half', 'cropped/lowerleft_quarter', 'cropped/lowerright_quarter', 'cropped/right_half', 'cropped/upper_half', 'cropped/upperleft_quarter', 'cropped/upperright_quarter', 'filtered/aden', 'filtered/inkwell', 'filtered/lofi', 'mirrored', 'rotated/45', 'rotated/90', 'rotated/180']

def get_edited_filepaths(original_filepath):
  filepaths = []
  original_basepath, original_filename = os.path.split(os.path.abspath(original_filepath))
  edited_basepath = original_basepath.replace('data_plain', 'data_prepared')
  edited_folders = get_edited_folders()
  for folder in edited_folders:
    filepaths.append(f'{edited_basepath}/{folder}/{original_filename}')
  return filepaths

os.chdir(original_basepath)

def make_pairs():
	# initialize two empty lists to hold the (image, image) pairs and
	# labels to indicate if a pair is positive or negative
  pairImages = []
  pairLabels = []

  # prepare a positive pair and update the images and labels
	# lists, respectively
  original_images = []
  sorted_files = sorted(os.listdir())
  for index, file in enumerate(sorted_files[:15]):
    if file.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
      try:
        img1 = Image.open(file).convert('RGB')
        original_images.append(file)
        print(file)
        edited_filepaths = get_edited_filepaths(file)
        for edited_file in edited_filepaths:
          img2 = Image.open(edited_file).convert('RGB')
          pairImages.append([img1, img2])
          pairLabels.append([1])
      except Exception as e:
        stdout.write(e)
    else:
      stdout.write(f"Skipping {os.path.split(os.path.abspath(file))[1]}...")

  print(len(original_images))
  original_image_paths = [original_basepath+'/'+file for file in original_images]
  # prepare a negative pair of images and update our lists
  neg_basepath = '/content/drive/My Drive/DL project/data_neg'
  os.chdir(neg_basepath)
  i = 0
  for index, file in enumerate(os.listdir()):
    if file.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
      try:
        if(i == 15) : i = 0
        img1 = Image.open(original_image_paths[i]).convert('RGB')
        img2 = Image.open(file).convert('RGB')
        print(original_image_paths[i], file)
        pairImages.append([img1, img2])
        pairLabels.append([0])
        i = i+1
      except Exception as e:
        stdout.write(e)
    else:
      stdout.write(f"Skipping {os.path.split(os.path.abspath(file))[1]}...")

  # return a 2-tuple of our image pairs and labels
  return (np.array(pairImages), np.array(pairLabels))

In [ ]:
pairImages, pairlabels = make_pairs()

B0GysW1pSRR.jpg
BBz7jURC0Rr.jpg
BFZB3ARRwRr.jpg
BG7_XO0JARr.jpg
BGOinDhn5rR.jpg
BJhtTBEBnrr.jpg
BKyiObwBOrr.jpg
BL9d3Q-gDRR.jpg
BMOer6tl6RR.jpg
BMPWR0Hh7rr.jpg
BOo0tBKBVRR.jpg
BOsfcOIh2RR.jpg
BQbKwa2BgRr.jpg
BQq79LMhHRR.jpg
BROOFmkg7Rr.jpg
15
/content/drive/My Drive/DL project/data_plain/r/r/B0GysW1pSRR.jpg Copy of BfV84njBzrr.jpg
/content/drive/My Drive/DL project/data_plain/r/r/BBz7jURC0Rr.jpg Copy of Bib0FB-nnRR.jpg
/content/drive/My Drive/DL project/data_plain/r/r/BFZB3ARRwRr.jpg Copy of BKyiObwBOrr.jpg
/content/drive/My Drive/DL project/data_plain/r/r/BG7_XO0JARr.jpg Copy of BFZB3ARRwRr.jpg
/content/drive/My Drive/DL project/data_plain/r/r/BGOinDhn5rR.jpg Copy of BJhtTBEBnrr.jpg
/content/drive/My Drive/DL project/data_plain/r/r/BJhtTBEBnrr.jpg Copy of BfsviIzn7Rr.jpg
/content/drive/My Drive/DL project/data_plain/r/r/BKyiObwBOrr.jpg Copy of BkEQv_iFsRR.jpg
/content/drive/My Drive/DL project/data_plain/r/r/BL9d3Q-gDRR.jpg Copy of BL9d3Q-gDRR.jpg
/content/drive/My Drive/DL project/da

<ipython-input-3-be74732e2c85>:64: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return (np.array(pairImages), np.array(pairLabels))
<ipython-input-3-be74732e2c85>:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (np.array(pairImages), np.array(pairLabels))


In [ ]:
len(pairlabels)

540

In [ ]:
len(pairImages)

540

In [ ]:
# Shuffling data
num_samples = len(pairlabels)

# Generate shuffled indices
shuffled_indices = np.random.permutation(num_samples)

# Use shuffled indices to shuffle both X and y
X_shuffled = pairImages[shuffled_indices]
y_shuffled = pairlabels[shuffled_indices]

In [ ]:
from sklearn.model_selection import train_test_split

# Choose the split ratios (adjust as needed)
train_size = 0.70
val_size = 0.15
test_size = 0.15

# Perform a single split
train_data, test_val_data, train_labels, test_val_labels = train_test_split(
    X_shuffled, y_shuffled, test_size=(val_size + test_size), random_state=42
)

val_data, test_data, val_labels, test_labels = train_test_split(
    test_val_data, test_val_labels, test_size=test_size / (val_size + test_size), random_state=42
)

# Print the sizes of each split
print("Train set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Train set size: 378
Validation set size: 81
Test set size: 81


### Model training and evaluation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

In [ ]:
class ImagePairResNet(nn.Module):
    def __init__(self, num_classes=1):
        super(ImagePairResNet, self).__init__()
        resnet = models.resnet34(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])

        # Fully connected layers with dropout
        self.fc1 = nn.Linear(512 * 2, 256)
        self.dropout = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x1, x2):
        x1 = self.features(x1)
        x2 = self.features(x2)
        x = torch.cat((x1.view(x1.size(0), -1), x2.view(x2.size(0), -1)), dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
# Custom dataset class for image pairs
class ImagePairDataset(Dataset):
    def __init__(self, imagePairs, labels, transform=None):
        self.image_pairs = imagePairs
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        pair = self.image_pairs[idx]
        image1, image2 = pair[0], pair[1]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)

        return image1, image2, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create datasets and dataloaders for training and validation
train_dataset = ImagePairDataset(train_data, train_labels, transform=transform)
val_dataset = ImagePairDataset(val_data, val_labels, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
model = ImagePairResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and validation loop
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        images1, images2, labels = batch
        optimizer.zero_grad()
        outputs = model(images1, images2)
        outputs = torch.sigmoid(outputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            images1, images2, labels = batch
            outputs = model(images1, images2)
            val_loss += criterion(outputs, labels.view(-1, 1)).item()

    val_loss /= len(val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item()}, Val Loss: {val_loss}')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 110MB/s]


Epoch 1/10, Train Loss: 0.6931053996086121, Val Loss: 4.314207474390666
Epoch 2/10, Train Loss: 0.7463862299919128, Val Loss: 82.89892959594727
Epoch 3/10, Train Loss: 0.6008244156837463, Val Loss: 6.453629811604817
Epoch 4/10, Train Loss: 0.6642294526100159, Val Loss: 4.352379719416301
Epoch 5/10, Train Loss: 0.7149127125740051, Val Loss: 12.551904996236166
Epoch 6/10, Train Loss: 0.6450513005256653, Val Loss: 2.9195220470428467
Epoch 7/10, Train Loss: 0.6360512375831604, Val Loss: 3.961162010828654
Epoch 8/10, Train Loss: 0.6073059439659119, Val Loss: 10.996488571166992
Epoch 9/10, Train Loss: 0.6492299437522888, Val Loss: 4.5985368092854815
Epoch 10/10, Train Loss: 0.7258924841880798, Val Loss: 12.10031795501709


In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/DL project/resnet34.pth')

In [ ]:
# Define transformations for test data (similar to training and validation transformations)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create dataset and dataloader for testing
test_dataset = ImagePairDataset(test_data, test_labels, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Testing loop
model.eval()
test_loss = 0.0
correct_predictions = 0

with torch.no_grad():
    for batch in test_dataloader:
        images1, images2, labels = batch
        outputs = model(images1, images2)
        test_loss += criterion(outputs, labels.view(-1, 1)).item()

        # Assuming binary classification, count correct predictions
        predicted_labels = (outputs >= 0.5).float()
        correct_predictions += (predicted_labels == labels.view(-1, 1)).sum().item()

test_loss /= len(test_dataloader)
accuracy = correct_predictions / len(test_dataset)

print(f'Test Loss: {test_loss}, Accuracy: {accuracy}')

Test Loss: 10.329416910807291, Accuracy: 0.5308641975308642
